# MAUDE & UDI 데이터 통합 탐색

이 노트북은 MAUDE와 UDI 데이터를 통합하여 탐색합니다.

## 목차
1. 환경 설정
2. 데이터 로딩
3. 유틸리티 함수 정의
4. MAUDE 데이터 탐색
5. UDI 데이터 탐색
6. MAUDE-UDI 매칭 분석

## 1. 환경 설정

### 라이브러리 불러오기

In [ ]:
from typing import Tuple, List, Dict, Any
import polars as pl
import pandas as pd
import sys
from pathlib import Path
import psutil
import re

# 상대 경로 설정
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# sys.path 설정
if str(PROJECT_ROOT) in sys.path:
    sys.path.remove(str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 src 모듈 캐시 제거
if 'src' in sys.modules:
    del sys.modules['src']

# 프로젝트 모듈 import
from src.loading import DataLoader
from src.utils.polars import (
    get_pattern_cols, 
    get_unique_by_cols_safe,
)
from src.preprocess.eda import analyze_null_values

### 시스템 리소스 확인

In [ ]:
# CPU 및 메모리 정보
cpu_core = psutil.cpu_count(logical=True)
mem = psutil.virtual_memory()

print(f"사용할 워커 수: {cpu_core}")
print(f"총 메모리: {mem.total / (1024**3):.2f} GB")
print(f"사용 가능 메모리: {mem.available / (1024**3):.2f} GB")
print(f"사용 중인 메모리: {mem.used / (1024**3):.2f} GB")
print(f"메모리 사용률: {mem.percent}%")

### 전역 변수 및 설정

In [ ]:
# MAUDE 데이터 컬럼 패턴
BASE_COLS = [
    'mdr_report_key', 'adverse_event_flag', 'product_problems',
    'product_problem_flag', 'date_of_event', 'date_report', 
    'date_received', 'device_date_of_manufacturer', 'event_type',
    'previous_use_code', 'single_use_flag', 'report_source_code',
    'reprocessed_and_reused_flag', 'date_facility_aware', 'report_date',
    'report_to_fda', 'date_report_to_fda', 'report_to_manufacturer',
    'date_report_to_manufacturer', 'event_location', 
    'date_manufacturer_received', 'manufacturer_link_flag',
    'date_added', 'date_changed', 'pma_pmn_number',
    'suppl_dates_fda_received', 'suppl_dates_mfr_received'
]

DEVICE_PATTERNS = [
    '_brand_name', '_udi_di', '_device_report_product_code', 
    '_model_number', '_expiration_date_of_device', '_device_age_text',
    '_device_operator', '_implant_flag', '_manufacturer_d_name', 
    '_openfda_device_class', '_openfda_device_name', '_generic_name'
]

PATIENT_PATTERNS = [
    '_patient_sequence_number', '_patient_age', '_patient_sex',
    '_patient_weight', '_patient_race', '_patient_problems',
    '_sequence_number_outcome'
]

MDR_TEXT_PATTERNS = ['_text', '_text_type_code']

# 날짜 컬럼
DATE_COLS = [
    'date_of_event', 'date_report', 'date_manufacturer_received', 
    'date_received', 'date_added', 'suppl_dates_fda_received', 
    'suppl_dates_mfr_received', 'device_date_of_manufacturer', 
    'device_0_expiration_date_of_device', 'date_changed'
]

# UDI 패턴
IDENTIFIER_PATTERNS = [
    r"^device_\d+_brand_name$",
    r"identifiers_\d+_id", 
    r"identifiers_\d+_issuing_agency", 
    r"identifiers_\d+_package_discontinue_date", 
    r"identifiers_\d+_package_status", 
    r"identifiers_\d+_package_type", 
    r"identifiers_\d+_quantity_per_package", 
    r"identifiers_\d+_type", 
    r"identifiers_\d+_unit_of_use_id"
]

# Polars 설정
POLARS_KWARGS = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}

ADAPTER = 'polars'

## 2. 데이터 로딩

### MAUDE 데이터 로딩

In [ ]:
maude_loader = DataLoader(
    start=2020,
    end=2025,
    output_file=DATA_DIR / 'bronze' / 'maude_raw.parquet',
    max_workers=4
)

maude_lf = maude_loader.load(adapter=ADAPTER, **POLARS_KWARGS)
maude_lf

### UDI 데이터 로딩

In [ ]:
udi_loader = DataLoader(
    name='udi',
    output_file=DATA_DIR / 'bronze' / 'udi_raw.parquet',
)

udi_lf = udi_loader.load(adapter=ADAPTER, **POLARS_KWARGS)
udi_lf

## 3. 유틸리티 함수 정의

In [ ]:
def overview_col(df: pl.LazyFrame, col: str):
    """컬럼의 고유값 개수 및 상위/하위 100개 값 출력"""
    nunique = df.select(
        pl.col(col).n_unique().alias(f'unique_{col}')
    ).collect().item()
    
    print(f'{col}의 고유 개수: {nunique}')
    
    unique_df = df.select(
        pl.col(col).unique().sort().head(100).alias(f'head_{col}'),
        pl.col(col).unique().sort().tail(100).alias(f'tail_{col}'),
    ).collect().to_pandas()
    
    display(unique_df)

In [ ]:
def get_use_cols(
    df: pl.LazyFrame,
    base_cols: List[str] = BASE_COLS,
    device_patterns: List[str] = DEVICE_PATTERNS,
    patient_patterns: List[str] = PATIENT_PATTERNS,
    mdr_text_patterns: List[str] = MDR_TEXT_PATTERNS
) -> Tuple[List[str], List[str], List[str], List[str]]:
    """MAUDE 데이터에서 분석할 컬럼 추출"""
    total_cols = df.collect_schema().names()
    device_cols = [col for col in total_cols if col.startswith('device_') and 
                any(pattern in col for pattern in device_patterns)]
    patient_cols = [col for col in total_cols if col.startswith('patient_') and 
                    any(pattern in col for pattern in patient_patterns)]
    mdr_text_cols = [col for col in total_cols if col.startswith('mdr_text_') and 
                    any(col.endswith(pattern) for pattern in mdr_text_patterns)]

    analysis_cols = base_cols + device_cols + patient_cols + mdr_text_cols
    analysis_cols = sorted(list(set(analysis_cols)), reverse=True)

    print(f"총 컬럼: {len(analysis_cols)}개")
    print(f"  - 기본 컬럼: {len(base_cols)}개")
    print(f"  - device_*: {len(device_cols)}개")
    print(f"  - patient_*: {len(patient_cols)}개")
    print(f"  - mdr_text_*: {len(mdr_text_cols)}개")

    return device_cols, patient_cols, mdr_text_cols, analysis_cols

In [ ]:
def filter_notna_rows(df: pl.LazyFrame, col: str) -> Tuple[pl.LazyFrame, pl.Expr, pl.DataFrame]:
    """결측값이 아닌 행만 필터링"""
    na_patterns = r'^None$|^UNK|^NOT APPLICABLE$|^N/A$|^NA$|^$|\s+$'
    notna_cond = True
    notna_cond &= ~pl.col(col).str.to_uppercase().str.contains(na_patterns)
    filtered = df.filter(notna_cond)
    notna_rows = filtered.select(pl.len()).collect()
    return filtered, notna_cond, notna_rows

In [ ]:
def summarize_date_gaps(
    lf: pl.LazyFrame,
    date_col1: str = None,
    date_col2: str = None,
    fmt: str = "%Y%m%d",
) -> pl.LazyFrame:
    """두 날짜 컬럼 간의 차이 통계 계산"""
    return (
        lf
        .select([date_col1, date_col2])
        .drop_nulls()
        .with_columns([
            pl.col(date_col1)
              .cast(pl.Utf8)
              .str.strptime(pl.Date, format=fmt, strict=False)
              .alias(date_col1),
            pl.col(date_col2)
              .cast(pl.Utf8)
              .str.strptime(pl.Date, format=fmt, strict=False)
              .alias(date_col2)
        ])
        .with_columns([
            (pl.col(date_col1).cast(pl.Int64) - pl.col(date_col2).cast(pl.Int64))
                .alias("days_date1_minus_date2"),
        ])
        .select([
            pl.col("days_date1_minus_date2").min().alias("min_date1_date2"),
            pl.col("days_date1_minus_date2").max().alias("max_date1_date2"),
            pl.col("days_date1_minus_date2").median().alias("med_date1_date2"),
            pl.col("days_date1_minus_date2").mean().alias("mean_date1_date2"),
        ])
    )

In [ ]:
def search_condition(field: str, value: str, exact=False) -> bool:
    """검색 조건 생성"""
    if exact:
        cond = pl.col(field).str.to_uppercase() == value
    else:
        cond = pl.col(field).str.to_uppercase().str.starts_with(value.upper())
    return cond

def search(
    lf: pl.LazyFrame, 
    query: List[Dict[str, Any]], 
    cols: List[str], 
    n_rows: int = 10, 
    transpose=False
) -> pd.DataFrame:
    """데이터 검색 및 필터링"""
    if not cols:
        cols = lf.collect_schema().names()
    
    cond = True
    for q in query:
        cond &= search_condition(q['field'], q['value'], q.get('exact', False))
        
    response_df = lf.select(cols).filter(cond).head(n_rows).collect().to_pandas()
    if transpose:
        response_df = response_df.transpose()
    return response_df

## 4. MAUDE 데이터 탐색

### 사용할 컬럼 추출

In [ ]:
device_cols, patient_cols, mdr_text_cols, analysis_cols = get_use_cols(maude_lf)

### 타입 및 고유값 확인

In [ ]:
# 첫 번째 device 컬럼들만
first_device_cols = [col for col in device_cols if col[7] == '0']

# 예시: 첫 번째 device 컬럼 확인
# for col in first_device_cols:
#     overview_col(maude_lf, col)

In [ ]:
# 첫 번째 patient 컬럼들만
first_patient_cols = [col for col in patient_cols if col[8] == '0']

# 예시: 첫 번째 patient 컬럼 확인
# for col in first_patient_cols:
#     overview_col(maude_lf, col)

In [ ]:
# 첫 번째 mdr_text 컬럼들
first_mdr_cols = [col for col in mdr_text_cols if '_0_' in col]

# 예시: 첫 번째 mdr_text 컬럼 확인
# for col in first_mdr_cols:
#     overview_col(maude_lf, col)

### 날짜 컬럼 분석

In [ ]:
# 날짜 컬럼 샘플 확인
maude_lf.select(DATE_COLS).head(10).collect().to_pandas()

In [ ]:
# 날짜 간격 분석 예시
date_col1 = 'date_added'
date_col2 = 'date_received'
date_gaps = summarize_date_gaps(maude_lf, date_col1, date_col2)
date_gaps.collect()

### 제품군 확인

In [ ]:
# 제품 코드별 generic_name 고유 개수
product_cols = [
    'device_0_brand_name', 
    'device_0_generic_name', 
    'device_0_device_report_product_code', 
    'device_0_openfda_device_name'
]

maude_lf.select(product_cols).group_by('device_0_device_report_product_code').agg(
    pl.col('device_0_generic_name').n_unique().alias('unique_generic_name'),
    pl.col('device_0_openfda_device_name').n_unique().alias('unique_device_name'),
).collect().to_pandas().sort_values('unique_generic_name', ascending=False).head(20)

### 결측치 분석

In [ ]:
# 분석 컬럼의 결측치 확인
null_df = analyze_null_values(maude_lf, analysis_cols)
null_df.head(20)

### UDI-DI 및 모델 번호 커버리지 분석

In [ ]:
total_rows = maude_lf.select(pl.len()).collect().item()

udi_filtered, udi_notna_cond, udi_notna_rows = filter_notna_rows(maude_lf, 'device_0_udi_di')
model_filtered, model_notna_cond, model_notna_rows = filter_notna_rows(maude_lf, 'device_0_model_number')

coverage_analysis = maude_lf.select([
    udi_notna_cond.alias("has_udi"),
    model_notna_cond.alias("has_model")
]).group_by(["has_udi", "has_model"]).len().with_columns([
    pl.format("{}%", (pl.col("len").mul(100).truediv(total_rows).round(1)))
])

actual_coverage = maude_lf.select([
    (udi_notna_cond | model_notna_cond).mean().alias("either"),
    (udi_notna_cond & model_notna_cond).mean().alias("both")
])

print("UDI-DI 및 모델 번호 커버리지:")
display(coverage_analysis.collect())
display(actual_coverage.collect())

### MAUDE 데이터의 모든 UDI-DI 수집

In [ ]:
# 모든 device_*_udi_di 컬럼에서 고유값 추출
device_udi_cols = [f'device_{i}_udi_di' for i in range(50)]

maude_udi_unique = set()
for col in device_udi_cols:
    if col in maude_lf.collect_schema().names():
        unique_vals = maude_lf.select(pl.col(col).unique()).collect().to_series().to_list()
        maude_udi_unique.update(unique_vals)

# None 제거
maude_udi_unique.discard(None)

print(f"MAUDE 데이터의 고유 UDI-DI 개수: {len(maude_udi_unique):,}")

## 5. UDI 데이터 탐색

### UDI 패턴 추출

In [ ]:
# identifiers 컬럼 패턴
udi_all_pattern = [r'^identifiers_\d+_id$']
udi_cols = get_pattern_cols(udi_lf, udi_all_pattern)
print(f"UDI 컬럼 개수: {len(udi_cols)}")

### UDI 고유값 추출

In [ ]:
# UDI 데이터의 모든 고유 UDI-DI
cols_group = {'udi': udi_cols}

udi_udi_unique = get_unique_by_cols_safe(
    udi_lf, 
    cols_group,
    memory_safety_ratio=0.3,
    calibration_factor=1
)['udi']

print(f"UDI 데이터의 고유 UDI-DI 개수: {len(udi_udi_unique):,}")

### Primary UDI-DI 분석

In [ ]:
# Type 컬럼 추출
type_patterns = [r'identifiers_\d+_type']
type_cols = get_pattern_cols(udi_lf, type_patterns)

# ID 컬럼 추출
id_patterns = [r'identifiers_\d+_id']
id_cols = get_pattern_cols(udi_lf, id_patterns)

# Type-ID 쌍 매칭
def extract_index(col_name):
    match = re.search(r'identifiers_(\d+)_', col_name)
    return int(match.group(1)) if match else None

type_id_pairs = []
for type_col in type_cols:
    idx = extract_index(type_col)
    id_col = f'identifiers_{idx}_id'
    if id_col in id_cols:
        type_id_pairs.append((type_col, id_col))

print(f"Type-ID 쌍 개수: {len(type_id_pairs)}")

In [ ]:
# Primary UDI-DI 추출
primary_udi_unique = set()

for i, (type_col, id_col) in enumerate(type_id_pairs):
    print(f"Processing {i+1}/{len(type_id_pairs)}: {type_col}...", end=" ")
    
    try:
        count = (
            udi_lf
            .filter(pl.col(type_col).eq("Primary"))
            .select(pl.len())
            .collect()
            .item()
        )
        
        if count > 0:
            ids = (
                udi_lf
                .filter(pl.col(type_col).eq("Primary"))
                .select(pl.col(id_col))
                .unique()
                .collect()
                .to_series()
                .drop_nulls()
                .to_list()
            )
            primary_udi_unique.update(ids)
            print(f"✓ Found {len(ids)} unique Primary IDs")
        else:
            print("✓ No Primary found")
            
    except Exception as e:
        print(f"✗ Error: {e}")
        continue

print(f"\n{'='*50}")
print(f"Total unique Primary IDs: {len(primary_udi_unique):,}")

In [ ]:
# Primary 컬럼 생성
udi_with_primary_lf = udi_lf.with_columns(
    pl.coalesce([
        pl.when(pl.col(type_col).eq("Primary"))
        .then(pl.col(id_col))
        for type_col, id_col in type_id_pairs
    ]).alias('primary_udi_di')
)

# 확인
udi_with_primary_lf.head(5).collect()

## 6. MAUDE-UDI 매칭 분석

### 매칭 통계

In [ ]:
# MAUDE와 UDI 전체 비교
angry_udi = maude_udi_unique - udi_udi_unique
survive_udi = maude_udi_unique & udi_udi_unique

print(f"UDI 데이터의 고유 UDI-DI 개수: {len(udi_udi_unique):,}")
print(f"MAUDE 데이터의 고유 UDI-DI 개수: {len(maude_udi_unique):,}")
print(f"UDI 데이터에 없는 MAUDE UDI-DI: {len(angry_udi):,}개")
print(f"UDI 데이터에 있는 MAUDE UDI-DI: {len(survive_udi):,}개")
print(f"매칭률: {len(survive_udi) / len(maude_udi_unique) * 100:.2f}%")

### Primary와 매칭

In [ ]:
# Primary IDs를 LazyFrame으로
primary_lf = pl.LazyFrame({
    'udi_di': list(primary_udi_unique)
})

maude_udi_lf = pl.LazyFrame({
    'udi_di': list(maude_udi_unique)
})

# 매칭되는 UDI-DI
maude_udi_matched = (
    maude_udi_lf
    .select('udi_di')
    .unique()
    .join(
        primary_lf,
        left_on='udi_di',
        right_on='udi_di',
        how='semi'
    )
    .collect()
    .to_series()
    .to_list()
)

# 매칭 안 되는 UDI-DI
maude_udi_unmatched = (
    maude_udi_lf
    .select('udi_di')
    .unique()
    .join(
        primary_lf,
        left_on='udi_di',
        right_on='udi_di',
        how='anti'
    )
    .collect()
    .to_series()
    .to_list()
)

print(f"Primary와 매칭되는 고유 MAUDE UDI-DI: {len(maude_udi_matched):,}")
print(f"Primary와 매칭되지 않는 고유 MAUDE UDI-DI: {len(maude_udi_unmatched):,}")
print(f"매칭률: {len(maude_udi_matched) / len(maude_udi_unique) * 100:.2f}%")

### 매칭 데이터 필터링

In [ ]:
# MAUDE 데이터를 매칭 여부로 분리
maude_unique_lf = maude_lf.select('device_0_udi_di').unique()

maude_udi_matched_lf = (
    maude_unique_lf
    .join(
        primary_lf,
        left_on='device_0_udi_di',
        right_on='udi_di',
        how='semi'
    )
)

maude_udi_unmatched_lf = (
    maude_unique_lf
    .join(
        primary_lf,
        left_on='device_0_udi_di',
        right_on='udi_di',
        how='anti'
    )
)

# 원본 데이터 join
maude_matched_lf = maude_lf.join(
    maude_udi_matched_lf,
    on='device_0_udi_di',
    how='semi'
)

maude_unmatched_lf = maude_lf.join(
    maude_udi_unmatched_lf,
    on='device_0_udi_di',
    how='semi'
)

print(f"전체 MAUDE 데이터: {maude_lf.select(pl.len()).collect().item():,}")
print(f"Primary와 매칭되는 MAUDE 데이터: {maude_matched_lf.select(pl.len()).collect().item():,}")
print(f"Primary와 매칭 안 되는 MAUDE 데이터: {maude_unmatched_lf.select(pl.len()).collect().item():,}")

### 스코핑 예시

In [ ]:
# 선택 컬럼 정의
udi_select_cols = ['company_name', 'brand_name', 'version_or_model_number', 'identifiers_0_id']
maude_select_cols = ['device_0_manufacturer_d_name', 'device_0_brand_name', 'device_0_model_number', 'device_0_udi_di']

# 샘플 UDI-DI로 검색
sample_udi_dis = sorted(maude_udi_matched)[:5]

for udi_di in sample_udi_dis:
    query = [
        {
            'field': 'device_0_udi_di',
            'value': udi_di,
            'exact': True
        }
    ]
    print(f"\n=== UDI-DI: {udi_di} ===")
    display(search(maude_lf, query, maude_select_cols, n_rows=5))

In [ ]:
# 제조사별 검색 예시
sample_companies = ['DEXCOM', 'ALCON']

for company in sample_companies:
    query = [
        {
            'field': 'company_name',
            'value': company
        }
    ]
    print(f"\n=== Company: {company} ===")
    display(search(udi_lf, query, udi_select_cols, n_rows=10).sort_values(by='brand_name'))